# Run Deeplabut for freely moving box

We will first import the libraries needed. If you get an error, check that you're running the notebook inside the environment where DeepLabCut is installed.

In [1]:
import deeplabcut
import os, glob, re, datetime, shutil
import dlc_fun

# we will set some general path identifiers here
datapath       = 'S:\\ElboustaniLab\\#SHARE\\Data'
jointmod       = '0Dyad_JointPerceptualDecisionMaking'
ffmpegpath     = 'C:\\ffmpeg\\bin'
filmidentifier = 'Filming'

Loading DLC 3.0.0rc8...


We will now specify the model for analysis. Only one model is available, for unimplanted mice.

In [2]:
project_twomice = 'S:\\ElboustaniLab\\#SHARE\\Analysis\\JointDecisionDeepLabCut\\TwoMiceTorch-DK-2025-04-15'
project_slider  = 'S:\\ElboustaniLab\\#SHARE\\Analysis\\JointDecisionDeepLabCut\\MouseSlider-DK-2025-04-15'
config_twomice  = os.path.join(project_twomice,'config.yaml')
config_slider   = os.path.join(project_slider,'config.yaml')

### Find videos

Specify the mouse or pair for which you want to perform the analysis. This string should match folder namings.

In [3]:
#mousefolders = ['YX018', 'YX019', 'YX020', 'YX021',  
#                'Slider_YX015','Slider_YX014',
#               'YX015', 'YX014', 'YX017', 'YX016']
mousefolders = ['YX015', 'YX014', 'YX017', 'YX016']
datestart    = '20220201'

In [4]:
mousepaths = []
for imouse in range(len(mousefolders)):
    if '_' in mousefolders[imouse]:
        mousepaths.append(os.path.join(datapath, jointmod, mousefolders[imouse]))
    else:
        mousepaths.append(os.path.join(datapath, mousefolders[imouse]))
        
for impath in range(len(mousepaths)):
    if os.path.isdir(mousepaths[impath]): 
        print(str(impath+1) + '. data path found: ' + mousepaths[impath])
    else:
        print(str(impath+1) + 'no such mouse or pair, check your list')


1. data path found: S:\ElboustaniLab\#SHARE\Data\0Dyad_JointPerceptualDecisionMaking\Slider_YX014


In [5]:
listvidpaths = []
for imfold in range(len(mousepaths)):
    allfpaths = dlc_fun.find_mp4_files(mousepaths[imfold])
    listvidpaths.extend(allfpaths)

In [6]:
# Create a list of tuples (date, path)
dated_paths = [(dlc_fun.extract_date(path), path) for path in listvidpaths]

# Filter out None dates and sort the list by date
sorted_paths = sorted((dp for dp in dated_paths if dp[0] is not None), key=lambda x: x[0])

datethres    = datetime.datetime.strptime(datestart, '%Y%m%d')

# Extract the sorted paths
sorted_paths_only = [path for date, path in sorted_paths 
                     if datetime.datetime.strptime(date, '%Y%m%d') >= datethres and 
                     ('Unsorted' not in path) and ('Habituation' not in path) and 
                     ('Observational' not in path) and  ('ToSort' not in path) and  ('Direction' not in path)]
 
# paths analyzed
pathsold = [path for path in sorted_paths_only if not dlc_fun.to_analyze_dlc(path)]

# clean up pickle and h5 pathsold
for ipath in range(len(pathsold)):
    dlc_fun.remove_big_files(os.path.splitext(pathsold[ipath])[0])
        
# paths to run DLC for
pathsrun = [path for path in sorted_paths_only if dlc_fun.to_analyze_dlc(path)]
pathsrun.reverse()

if (len(sorted_paths_only)==0):
    print('No video folder found.')
else:
    print('The following files were found that have not been analyzed:')
    for ivid in range(len(pathsrun)):
        print(pathsrun[ivid])
       

The following files were found that have not been analyzed:
S:\ElboustaniLab\#SHARE\Data\0Dyad_JointPerceptualDecisionMaking\Slider_YX014\Filming\SliderSingleMouse\20250411\Session1\20250411_1851_Slider_YX014_1_reduced.mp4
S:\ElboustaniLab\#SHARE\Data\0Dyad_JointPerceptualDecisionMaking\Slider_YX014\Filming\SliderSingleMouse\20250410\Session1\20250410_1916_Slider_YX014_1_reduced.mp4
S:\ElboustaniLab\#SHARE\Data\0Dyad_JointPerceptualDecisionMaking\Slider_YX014\Filming\SliderSingleMouse\20250409\Session1\20250409_1938_Slider_YX014_2_reduced.mp4


### Run model

Let's run deeplabcut now.

In [7]:
for ipath in range(len(pathsrun)):
    
    currpath = pathsrun[ipath]
    # find if we can crop video
    isslider = 'Slider' in currpath
    ispair   = (jointmod in currpath) and (not isslider)
    skipcrop = ispair or ('Visual' in currpath)
    
    if isslider:
        config = config_slider
        nt     = 1
    else:
        config = config_twomice
        nt     = 2
    
    if "reduced" in currpath.lower():
        # video is already compressed by us
        localpath = dlc_fun.copy_video_locally(currpath, 'C:\\Temp_proc')
    else:
        # we compress
        localpath, status = dlc_fun.ffmpeg_compress_mp4_video(currpath, 'C:\\Temp_proc')
                                                  
    # do cropping
    if skipcrop:
        cropval = None
    else:
        cropval = dlc_fun.get_mouse_compartment(localpath, isslider)
        
    # run DLC locally
    deeplabcut.analyze_videos(config, localpath, 
                              videotype = 'mp4', 
                              cropping = cropval,
                              auto_track = True, 
                              n_tracks = nt, 
                              save_as_csv = True, 
                              batchsize = 4)
    
    csvpath = glob.glob(os.path.splitext(localpath)[0] + '*_el.csv')
    
    # if video was cropped, fix csv file
    if not skipcrop:
        dlc_fun.update_saved_csv(csvpath[0], cropval[2])

    # move dlc csv back to path
    shutil.copy2(csvpath[0], os.path.split(currpath)[0])
    
    # remove h5 and pickles after you're done, and other local files
    dlc_fun.remove_big_files(os.path.splitext(localpath)[0])
    os.remove(localpath)
    os.remove(csvpath[0])

Copying file to C:\Temp_proc\20250411_1851_Slider_YX014_1_reduced.mp4
cropping for slider, assering slider is on TOP...
Analyzing videos with S:\ElboustaniLab\#SHARE\Analysis\JointDecisionDeepLabCut\MouseSlider-DK-2025-04-15\dlc-models-pytorch\iteration-2\MouseSliderApr15-trainset95shuffle1\train\snapshot-best-125.pt
Starting to analyze C:\Temp_proc\20250411_1851_Slider_YX014_1_reduced.mp4
Video metadata: 
  Overall # of frames:    478927
  Duration of video [s]:  15964.23
  fps:                    30.0
  resolution:             w=1408, h=1197

Running pose prediction with batch size 4


  0%|                                                                          | 279/478927 [00:39<18:38:20,  7.13it/s]


KeyboardInterrupt: 